
# ファイルI/O

これまで、データを処理する方法と、深層学習モデルを構築、トレーニング、テストする方法について説明しました。ただし、ある時点で、学習されたモデルに十分満足できるようになり、後でさまざまなコンテキストで使用できるように結果を保存したいと考えます (おそらく、デプロイメントで予測するためでもあります)。さらに、長いトレーニング プロセスを実行する場合、サーバーの電源コードにつまずいても数日分の計算が失われないように、中間結果 (チェックポイント) を定期的に保存することがベスト プラクティスです。したがって、個々の重みベクトルとモデル全体の両方をロードおよび保存する方法を学習するときが来ました。このセクションでは両方の問題について説明します。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F


## ( **Tensor のロードと保存**)

個々のテンソルについては、 `load`関数と`save`関数を直接呼び出して、それぞれ読み取りおよび書き込みを行うことができます。どちらの関数も名前を指定する必要があり、 `save`保存する変数を入力として要求します。


In [2]:
x = torch.arange(4)
torch.save(x, 'x-file')


これで、保存されたファイルからデータをメモリに読み取ることができます。


In [3]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])


[**テンソルのリストを保存し、それらをメモリに読み戻すことができます。** 】


In [4]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))


[**文字列からテンソルにマッピングする辞書を書いたり読んだりすることもできます。** ] これは、モデル内のすべての重みを読み書きしたい場合に便利です。


In [5]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}


## [**モデルパラメータのロードと保存**]

個々の重みベクトル (または他のテンソル) を保存することは便利ですが、モデル全体を保存 (後でロード) する場合は非常に面倒になります。結局のところ、何百ものパラメータ グループが全体に散在している可能性があります。このため、深層学習フレームワークには、ネットワーク全体をロードおよび保存するための組み込み機能が提供されています。注意すべき重要な点は、これによりモデル全体ではなくモデル*パラメーターが*保存されるということです。たとえば、3 層 MLP がある場合、アーキテクチャを個別に指定する必要があります。その理由は、モデル自体に任意のコードが含まれる可能性があるため、モデルを自然にシリアル化できないためです。したがって、モデルを復元するには、コードでアーキテクチャを生成し、ディスクからパラメータをロードする必要があります。 (**おなじみの MLP から始めましょう。** )


In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)


次に、「mlp.params」という名前で [**モデルのパラメーターをファイルとして保存**] します。


In [7]:
torch.save(net.state_dict(), 'mlp.params')


モデルを復元するには、元の MLP モデルのクローンをインスタンス化します。モデルパラメータをランダムに初期化する代わりに、[**ファイルに保存されているパラメータを直接読み取ります**]。


In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)


両方のインスタンスが同じモデル パラメーターを持っているため、同じ入力`X`の計算結果は同じになるはずです。これを確認してみましょう。


In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])


## まとめ

`save`および`load`関数を使用して、テンソル オブジェクトのファイル I/O を実行できます。パラメータ辞書を介して、ネットワークのパラメータのセット全体を保存およびロードできます。アーキテクチャの保存は、パラメーターではなくコードで行う必要があります。

## 演習
1. トレーニング済みモデルを別のデバイスにデプロイする必要がない場合でも、モデル パラメーターを保存することの実際的な利点は何でしょうか?
1. ネットワークの一部のみを再利用して、別のアーキテクチャのネットワークに組み込むとします。たとえば、前のネットワークの最初の 2 つのレイヤーを新しいネットワークで使用するにはどうすればよいでしょうか?
1. ネットワーク アーキテクチャとパラメータを保存するにはどうすればよいでしょうか?アーキテクチャにどのような制限を課しますか?



[ディスカッション](https://discuss.d2l.ai/t/61)
